In [1]:
import glob
import os
import shutil
from tqdm import tqdm

ModuleNotFoundError: No module named 'tqdm'

GTFS（バスの処理）

In [ ]:
target_jcodes = [
"14101",
"14102",
"14103",
"14104",
"14105",
"14106",
"14107",
"14108",
"14109",
"14110",
"14111",
"14112",
"14113",
"14114",
"14115",
"14116",
"14117",
"14118"
]
target_jcode = "14100"

In [5]:
target_paths = []
for path in glob.glob("Y:/GTFS/Bus_Japan_Gov/*"):
    if path.split("\\")[-1] in target_jcodes:
        target_paths.append(path)

In [6]:
target_paths

['Y:/GTFS/Bus_Japan_Gov\\14101',
 'Y:/GTFS/Bus_Japan_Gov\\14102',
 'Y:/GTFS/Bus_Japan_Gov\\14103',
 'Y:/GTFS/Bus_Japan_Gov\\14104',
 'Y:/GTFS/Bus_Japan_Gov\\14105',
 'Y:/GTFS/Bus_Japan_Gov\\14106',
 'Y:/GTFS/Bus_Japan_Gov\\14107',
 'Y:/GTFS/Bus_Japan_Gov\\14108',
 'Y:/GTFS/Bus_Japan_Gov\\14109',
 'Y:/GTFS/Bus_Japan_Gov\\14110',
 'Y:/GTFS/Bus_Japan_Gov\\14111',
 'Y:/GTFS/Bus_Japan_Gov\\14112',
 'Y:/GTFS/Bus_Japan_Gov\\14113',
 'Y:/GTFS/Bus_Japan_Gov\\14114',
 'Y:/GTFS/Bus_Japan_Gov\\14115',
 'Y:/GTFS/Bus_Japan_Gov\\14116',
 'Y:/GTFS/Bus_Japan_Gov\\14117',
 'Y:/GTFS/Bus_Japan_Gov\\14118']

In [12]:
# 保存先ディレクトリ
output_dir = f'Y:/GTFS/Bus_Japan_Gov/additional/{target_jcode}'

# 保存先フォルダが存在しない場合、作成する
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 保存済みのファイル名を格納するセット
seen_files = set()

# 各ディレクトリを巡回し、.zipファイルを探す
for dir_path in tqdm(target_paths):
    if os.path.exists(dir_path):
        for file_name in os.listdir(dir_path):
            if file_name.endswith('.zip'):
                # ファイル名が重複していないか確認
                if file_name not in seen_files:
                    # 新しいファイルの場合、セットに追加してコピー
                    seen_files.add(file_name)
                    full_path = os.path.join(dir_path, file_name)
                    shutil.copy(full_path, os.path.join(output_dir, file_name))

100%|██████████| 18/18 [00:57<00:00,  3.18s/it]


GTFS（鉄道）の処理

In [16]:
import pandas as pd
import geopandas as gpd
import sqlite3
from tqdm import tqdm
import numpy as np

import os
import zipfile

def zip_text_files(directory_path, output_zip_file):
    # 新しいZIPファイルを作成
    with zipfile.ZipFile(output_zip_file, 'w') as zipf:
        # 指定ディレクトリのファイルをループ
        for foldername, subfolders, filenames in os.walk(directory_path):
            for filename in filenames:
                if filename.endswith('.txt'):
                    # ファイルパスを取得
                    file_path = os.path.join(foldername, filename)
                    # ZIPファイルにファイルを追加
                    zipf.write(file_path, arcname=filename)
                    # print(f'Added {filename} to {output_zip_file}')

In [14]:
DIR_RAILWAY_JP = "Z:/lab/accessibility/Railway_Japan/"
TARGET_JCODE = "14100"

In [22]:
conn = sqlite3.connect(DIR_RAILWAY_JP + "StationPortDatabase.db")
# station = pd.read_sql_query(sql="select * from stationHasGeom",con=conn)
cursor = conn.cursor()
cursor.execute(f"select * from stationHasGoverment WHERE jcode IN {tuple(target_jcodes)}")
Station_Ids = []
for r in cursor.fetchall():
    Station_Ids.append(r[0])

cursor.execute(f"select * from stationHasGeom")
station_jp = cursor.fetchall()

conn.close()

In [24]:
conn = sqlite3.connect("C:/lab/accessibility/Railway_Japan/TripDataAllJapanRail.db")
# station = pd.read_sql_query(sql="select * from stationHasGeom",con=conn)
cursor = conn.cursor()

# cursor.execute(f"select * from sqlite_master")
# for r in cursor.fetchall():
#     print(r)

cursor.execute(f"select * from stop_times")
stop_times = cursor.fetchall()
cursor.execute(f"select * from routes")
routes = cursor.fetchall()
cursor.execute(f"select * from trips")
trips = cursor.fetchall()
# station_yawata = pd.read_sql_query(sql=f"select station_id,station_name from stationHasGoverment WHERE jcode = {YAWATA_JCODE}",con = conn)
conn.close()

In [25]:
Target_Trip_Ids = []
for stop_time in tqdm(stop_times):
    if stop_time[3] in Station_Ids:
        Target_Trip_Ids.append(stop_time[0])
Target_Trip_Ids = set(Target_Trip_Ids)

STOP_TIMES = []
for stop_time in tqdm(stop_times):
    if stop_time[0] in Target_Trip_Ids:
        stop_time = list(stop_time)
        if stop_time[1] != "":
            stop_time[1] = stop_time[1]+":00"
        if stop_time[2] != "":
            stop_time[2] = stop_time[2]+":00"
        stop_time = tuple(stop_time)
        STOP_TIMES.append(stop_time)

TRIPS = []
for trip in trips:
    if trip[0] in Target_Trip_Ids:
        TRIPS.append(trip)

Route_Ids = []
for trip in trips:
    if trip[0] in Target_Trip_Ids:
        Route_Ids.append(trip[1])
Route_Ids = set(Route_Ids)
ROUTES = []
for route in routes:
    if route[0] in Route_Ids:
        ROUTES.append(route)

Stop_Ids = []
for stop_time in tqdm(stop_times):
    if stop_time[0] in Target_Trip_Ids:
        Stop_Ids.append(stop_time[3])
Stop_Ids = set(Stop_Ids)

STOPS = []
for stops in station_jp:
    if stops[0] in Stop_Ids:
        STOPS.append(stops)

100%|██████████| 5710504/5710504 [00:01<00:00, 4747815.51it/s]


In [26]:
# GTFSデータの作成
DIR_OUT_GTFS = "Z:\\lab\\accessibility\\Railway_Japan\\regional\\"

Agency_id       = TARGET_JCODE +"-RAIL"
os.makedirs(DIR_OUT_GTFS+f"{Agency_id}.gtfs",exist_ok=True) # ディレクトリ作成
Agency_name     = "__"
Agency_url      = "https:/hoge/"
Agency_timezone = "Asia/Tokyo"
Agency_lang     = "ja"
pd.DataFrame({"agency_id":[Agency_id],"agency_name":[Agency_name],"agency_url":[Agency_url],"agency_timezone":[Agency_timezone],"agency_lang":[Agency_lang]}).to_csv(DIR_OUT_GTFS+f"{Agency_id}.gtfs/agency.txt",index=False,sep=",")

"""calendar.txt"""
Start_date = 20240401
End_date = 20250401
calendar_txt = {
    "service_id":["Weekday","Saturday","Holiday"],
    "monday":[1,0,0],
    "tuesday":[1,0,0],
    "wednesday":[1,0,0],
    "thursday":[1,0,0],
    "friday":[1,0,0],
    "saturday":[0,1,0],
    "sunday":[0,0,1],
    "start_date":[Start_date,Start_date,Start_date],
    "end_date":[End_date,End_date,End_date]
}
pd.DataFrame(calendar_txt).to_csv(DIR_OUT_GTFS+f"/{Agency_id}.gtfs/calendar.txt",index=False,sep=",")

"""stops.txt"""
pd.DataFrame(np.array(STOPS)[:,[0,1,3,4]],columns=["stop_id","stop_name","stop_lat","stop_lon"]).to_csv(DIR_OUT_GTFS+f"/{Agency_id}.gtfs/stops.txt",index=False,sep=",")

"""routes.txt"""
pd.DataFrame(ROUTES,columns=["route_id","route_short_name","route_type"]).to_csv(DIR_OUT_GTFS+f"/{Agency_id}.gtfs/routes.txt",index=False,sep=",")

"""trips.txt"""
pd.DataFrame(TRIPS,columns=["trip_id","route_id","service_id","trip_short_name"]).to_csv(DIR_OUT_GTFS+f"/{Agency_id}.gtfs/trips.txt",index=False,sep=",")

"""stop_times.txt"""
pd.DataFrame(STOP_TIMES,columns=["trip_id","arrival_time","departure_time","stop_id","stop_sequence"]).to_csv(DIR_OUT_GTFS+f"/{Agency_id}.gtfs/stop_times.txt",index=False,sep=",")

"""zip化"""
# ディレクトリパスと出力するZIPファイルの名前
directory_path  = DIR_OUT_GTFS+f"/{Agency_id}.gtfs"
output_zip_file = DIR_OUT_GTFS+f"/{Agency_id}.gtfs.zip"
# 関数を呼び出してZIP圧縮を実行
zip_text_files(directory_path, output_zip_file)